In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import librosa
import os
#from python_speech_features import mfcc
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv2D, Flatten, concatenate, Input, Dropout, MaxPooling2D
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import re
import pickle
import whisper
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import json

C:\Users\josun\anaconda3\envs\newbie\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
librosa.__version__

'0.9.0'

## JSON에서 텍스트 추출

In [239]:
def jsonText(json_directory):
    json_files = sorted(os.listdir(json_directory))
    
    emotion = ['기쁨', '놀람', '중립', '공포', '혐오', '분노', '슬픔']

    emotion_to_array = {
        '기쁨': np.array([1,0,0,0,0,0,0]),
        '놀람': np.array([0,1,0,0,0,0,0]),
        '중립': np.array([0,0,1,0,0,0,0]),
        '공포': np.array([0,0,0,1,0,0,0]),
        '혐오': np.array([0,0,0,0,1,0,0]),
        '분노': np.array([0,0,0,0,0,1,0]),
        '슬픔': np.array([0,0,0,0,0,0,1])
    }
    
    texts = {'combined_text':[]}
    label = []
    for file in json_files:
        jfile = os.path.join(json_directory, file)
        with open(jfile, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

        lb = data["화자정보"]["Emotion"]
        if lb == "Anxious" or lb == "Hurt" or lb == "Embarrassed":
            continue
        label.append(emotion_to_array[lb])
        text_content = data["전사정보"]["TransLabelText"]
        #text_content = np.array(text_content)
        texts['combined_text'].append(text_content)
        
    print(len(texts['combined_text']))
    print(label[1])
    return texts, label

In [913]:
json_dts = "D:\\Speech synthesis data by emotion and speech style\\data\\1.Training\\labeling\\TL2\\7.중립"
json_dts_sort = sorted(os.listdir(json_dts))

texts = {"combined_text":[]}
labels = []
for i in range(0, 5):
    json_directory = os.path.join(json_dts, json_dts_sort[i])
    print(json_directory)
    text, label = jsonText(json_directory)
    
    texts['combined_text'].extend(text['combined_text'])
    labels = labels + label
    
print("\n", len(texts))
print(len(texts['combined_text']))

print(len(labels))

D:\Speech synthesis data by emotion and speech style\data\1.Training\labeling\TL2\7.중립\0002_G1A4E7S0C0_LYT
1990
[0 0 1 0 0 0 0]
D:\Speech synthesis data by emotion and speech style\data\1.Training\labeling\TL2\7.중립\0003_G1A4E7S0C0_JCH
1987
[0 0 1 0 0 0 0]
D:\Speech synthesis data by emotion and speech style\data\1.Training\labeling\TL2\7.중립\0005_G1A3E7S0C0_LJB
2286
[0 0 1 0 0 0 0]
D:\Speech synthesis data by emotion and speech style\data\1.Training\labeling\TL2\7.중립\0006_G1A3E7S0C0_LHJ
1992
[0 0 1 0 0 0 0]
D:\Speech synthesis data by emotion and speech style\data\1.Training\labeling\TL2\7.중립\0008_G1A3E7S0C0_JYC
1992
[0 0 1 0 0 0 0]

 1
10247
10247


## 텍스트 특징 추출

In [331]:
#텍스트
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertModel.from_pretrained("monologg/kobert")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def add_kobert_embeddings(df, text_column):
    embeddings_numpy = []
    for text in df[text_column]:
        tokens = tokenizer.tokenize(text)
        tokens = ["[CLS]"] + tokens + ["[SEP]"]
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        tokens_tensor = torch.tensor([token_ids])
        with torch.no_grad():
            model.to(device)
            tokens_tensor.to(device)
            outputs = model(tokens_tensor)
            sentence_embedding = outputs[0][:, 0, :].squeeze().cpu().numpy()
        embeddings_numpy.append(sentence_embedding)
    df['kobert_embeddings_numpy'] = embeddings_numpy
    return df

In [914]:
df = add_kobert_embeddings(texts, 'combined_text')

# 리스트에 있는 숫자의 인덱스에 해당하는 값들을 가져와서 리스트로 만듦
text_features = df['kobert_embeddings_numpy']

print(len(text_features))
print(len(text_features[0]))

10247
768


## 오디오 특징 추출

In [278]:
def audioFeature(audio_directory):
    audio_files = sorted(os.listdir(audio_directory))
    
    audio_df = pd.DataFrame(columns=['feature'])
    i = 0
    for filename in audio_files:
        if filename.endswith(".wav"):
            audio_path = os.path.join(audio_directory, filename)
            X, sample_rate = librosa.load(audio_path,duration=1,sr=22050*2)
            sample_rate = np.array(sample_rate)
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=25), axis=0)
            feature = mfccs
            audio_df.loc[i] = [feature]
            i=i+1

    scaler = StandardScaler()

    audio_df2 = pd.DataFrame(columns=['normalized'])
    for k in range(len(audio_df)):
        features = audio_df.loc[k, 'feature'].reshape(1, -1)
        features_transposed = features.T

        # 정규화 적용
        scaled_features_transposed = scaler.fit_transform(features_transposed)

        # 다시 전치하여 shape을 (n_samples, n_features)로 변경
        scaled_features = scaled_features_transposed.T
        scaled_features = scaled_features.flatten()
        scaled_features.tolist()
        audio_df2.loc[k, 'normalized'] = scaled_features.tolist()

    audio_3 = pd.DataFrame(audio_df2['normalized'].values.tolist())
    audio_3 = audio_3.fillna(0)
    
    # 모든 행을 배열로 변환
    audio_features = []
    for index, row in audio_3.iterrows():
        row_array = np.array(row)
        audio_features.append(row_array)
        
    print(len(audio_features))
    
    return audio_features

In [857]:
audio_dts = "D:\\Speech synthesis data by emotion and speech style\\data\\1.Training\\source\\TS1\\3.분노"
audio_dts_sort = sorted(os.listdir(audio_dts))

audio_features = []


for i in range(20, 25):
    audio_directory = os.path.join(audio_dts, audio_dts_sort[i])
    print(audio_directory)
    audio_ft = audioFeature(audio_directory)
    
    audio_features += audio_ft

print("\n", len(audio_features))
print(len(audio_features[0]))

D:\Speech synthesis data by emotion and speech style\data\1.Training\source\TS1\3.분노\0025_G2A4E3S0C0_GJY
1990
D:\Speech synthesis data by emotion and speech style\data\1.Training\source\TS1\3.분노\0026_G2A3E3S0C0_JSH
1988
D:\Speech synthesis data by emotion and speech style\data\1.Training\source\TS1\3.분노\0027_G2A3E3S0C0_KYK
1990
D:\Speech synthesis data by emotion and speech style\data\1.Training\source\TS1\3.분노\0028_G2A3E3S0C0_KJS
1988
D:\Speech synthesis data by emotion and speech style\data\1.Training\source\TS1\3.분노\0029_G2A4E3S0C0_KJE
1989

 9945
87


## 특징 저장

In [617]:
# 파일 경로 지정
file_path1 = 'D:\\text_audio_features\\기쁨\\textF6.pkl'
file_path2 = "D:\\text_audio_features\\기쁨\\audioF6.pkl"
file_path3 = "D:\\text_audio_features\\기쁨\\y6.pkl"

# 파일에 변수 저장
with open(file_path1, 'wb') as file:
    pickle.dump(text_features, file)
    
with open(file_path2, 'wb') as file:
    pickle.dump(audio_features, file)
    
with open(file_path3, 'wb') as file:
    pickle.dump(labels, file)
    

In [936]:
import pickle
# 저장된 변수를 다시 불러오는 코드
with open('D:\\text_audio_features\\기쁨\\textF1.pkl', 'rb') as file:
    textF = pickle.load(file)

    # 저장된 변수를 다시 불러오는 코드
with open('D:\\text_audio_features\\기쁨\\audioF1.pkl', 'rb') as file:
    audioF = pickle.load(file)

# 저장된 변수를 다시 불러오는 코드
with open('D:\\text_audio_features\\기쁨\\y1.pkl', 'rb') as file:
    y = pickle.load(file)

print("textF:", len(textF))
print("audioF:", len(audioF))
print("y:", len(y))


textF: 10392
audioF: 10392
y: 10392
